In [ ]:
# !pip install transformers
# !pip install datasets
# !pip install -q sentencepiece
# !pip install rouge-score

In [21]:
import os
import re
import tensorflow as tf

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')
data_dir = "/content/gdrive/MyDrive/Classes/W266_NLP/w266_reddit_summarization/data/tensorflow_datasets/reddit/1.0.0"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
os.chdir(data_dir)
file_names = os.listdir(data_dir)
metadata_files = [i for i in file_names if re.search('^(?!reddit-train)', i)]
train_records = [i for i in file_names if re.search('^reddit-train', i)]

In [25]:
def parse_file(serialized_example):

  file_scruct = {
      'author': tf.io.FixedLenFeature([], tf.string),
      'body': tf.io.FixedLenFeature([], tf.string),
      'content': tf.io.FixedLenFeature([], tf.string),
      'id': tf.io.FixedLenFeature([], tf.string),
      'normalizedBody': tf.io.FixedLenFeature([], tf.string),
      'subreddit': tf.io.FixedLenFeature([], tf.string),
      'subreddit_id': tf.io.FixedLenFeature([], tf.string),
      'summary': tf.io.FixedLenFeature([], tf.string),
  }

  example1 = tf.io.parse_single_example(serialized_example, file_scruct)
  return example1

# parse:
dataset = tf.data.TFRecordDataset(train_records[:5]).map(parse_file)
data_dict = {'id': [], 'author': [], 'subreddit': [], 'content': [], 'summary': []}
keys_to_parse = ['id', 'author', 'subreddit', 'content', 'summary']
for i, item in enumerate(dataset):
  for key1 in keys_to_parse:
    data_dict[key1].append(item[key1])

In [27]:
data_dict['content'][0].numpy().decode()

'Here is something Olympic middle distance runner Nick Willis wrote about fellow Kiwi Sean Adams (Pitt) \n New Zealand has only ever had two players reach the lofty heights of the NBA. Sean marks, a 6”11 center, was the 44th draft pick in the 1998 NBA draft, and went on to play for six different teams over his 11 season career. Kirk Penny, the former New Zealand Breakers star, went un-drafted in 2003, but had a taste of the NBA game in small stints with the Miami Heat and Los Angeles Clippers. \n Basketball fans in New Zealand look once again to have a kiwi to cheer for on the American hardwood. Rotorua native, Steven Adams, has chosen to forgo his remaining three seasons of eligibility in the NCAA as a 7ft center for Pittsburgh, and declare for this years NBA draft. Pittsburgh fans were hearing whispers of the possibility of Adams being a “one and done” prospect when he first arrived on campus, but as his first season progressed, there was a sense around the program that he may choose